# 서울 입학생 현황

In [1]:
import pandas as pd
import re
import glob

all_data = []  # 여러 연도 데이터를 담을 리스트

# 파일 경로 패턴 지정
file_list = glob.glob('./data/*년도_입학생 현황(초)_서울특별시교육청.csv')

for filename in file_list:
    # 파일 읽기
    df = pd.read_csv(filename)
    
    # 파일 이름에서 연도 추출
    year_match = re.search(r'(\d{4})', filename)
    if year_match:
        df['년도'] = int(year_match.group(1))  # '년도' 컬럼 추가
        df = df[['년도','지역','정보공시 학교코드', '학교명','계-남', '계-여', '합계']].rename(columns={'계-남':'입학생_남', '계-여':'입학생_여', '합계':'입학생수'})
    all_data.append(df)  # 리스트에 추가

# 모든 데이터를 하나의 DataFrame으로 합치기
final_data = pd.concat(all_data, ignore_index=True)

# 학교의 학생 수

In [ ]:
all_data = []  # 모든 DataFrame을 담을 리스트

# 파일 경로 패턴
file_list = glob.glob('./data/*년도_학년별·학급별 학생수(초)_서울특별시교육청.csv')

for filename in file_list:
    # 파일 읽기
    df1 = pd.read_csv(filename)
    
    # 필요 없는 컬럼 제거
    df1.drop(columns=['학교급코드', '설립구분', '제외여부','지역',
                     '제외사유','시도교육청', '교육지원청','순회학급 학급수', '순회학급 학생수',
       '순회학급 학급당 학생수'], inplace=True)
    
    # 파일 이름에서 연도 추출 후 '년도' 컬럼 추가
    year_match = re.search(r'(\d{4})', filename)
    if year_match:
        df1['년도'] = int(year_match.group(1))
    
    all_data.append(df1)  # 리스트에 추가

# 모든 데이터를 하나의 DataFrame으로 합치기
final_data_all = pd.concat(all_data, ignore_index=True)

In [4]:
# 예시: inner join (세 컬럼 모두 일치하는 행만 남김)
merged_data = pd.merge(
    final_data, 
    final_data_all,
    on=['년도', '정보공시 학교코드', '학교명'],
    how='inner'  # inner, left, right, outer 가능
)

merged_data.columns

Index(['년도', '지역', '정보공시 학교코드', '학교명', '입학생_남', '입학생_여', '입학생수', '1학년 학급수',
       '1학년 학생수', '1학년 학급당 학생수', '2학년 학급수', '2학년 학생수', '2학년 학급당 학생수',
       '3학년 학급수', '3학년 학생수', '3학년 학급당 학생수', '4학년 학급수', '4학년 학생수',
       '4학년 학급당 학생수', '5학년 학급수', '5학년 학생수', '5학년 학급당 학생수', '6학년 학급수',
       '6학년 학생수', '6학년 학급당 학생수', '특수학급 학급수', '특수학급 학생수', '특수학급 학급당 학생수',
       '학급수(계)', '학생수(계)', '학급당 학생수(계)', '교사수', '수업교원 1인당 학생수'],
      dtype='object')

In [6]:
## 두 프레임 합쳐서 저장

In [5]:
merged_data.to_csv('data_all.csv', index=False, encoding='utf-8-sig')

## 폐교학교의 지정

In [ ]:
years = sorted(merged_data['년도'].unique())
last_year = years[-1]  # 2025년

closed_list = []

for year in years[:-1]:  # 마지막 연도 제외
    data_curr = merged_data[merged_data['년도'] == year].reset_index(drop=True)
    
    for _, row in data_curr.iterrows():
        school_code = row['정보공시 학교코드']
        
        # 해당 학교가 이후 데이터에 한 번도 등장하지 않으면 폐교로 간주
        appears_later = merged_data[
            (merged_data['년도'] > year) & (merged_data['정보공시 학교코드'] == school_code)
        ]
        
        if appears_later.empty:  # 이후에 안 나타나면 폐교
            closed_school = row.copy()
            closed_school['폐교연도'] = year + 1  # 최초로 사라진 다음 연도
            closed_list.append(closed_school)

# 결과 DataFrame 생성
c = pd.DataFrame(closed_list)
print(c[['년도', '폐교연도', '정보공시 학교코드', '학교명']])

       년도  폐교연도   정보공시 학교코드        학교명
225  2014  2015  S010001256   서울흥일초등학교
422  2014  2015  S010002020  알로이시오초등학교
421  2017  2018  S010002022     은혜초등학교
146  2019  2020  S010001047   서울염강초등학교
16   2022  2023  S010000754   서울반포초등학교
454  2022  2023  S010002195   서울화양초등학교
234  2023  2024  S010001407   서울면중초등학교
510  2024  2025  S010002379   서울보광초등학교


In [13]:
closed_schools_all

,년도,지역,정보공시 학교코드,학교명,입학생_남,입학생_여,입학생수,1학년 학급수,1학년 학생수,1학년 학급당 학생수,...,6학년 학급당 학생수,특수학급 학급수,특수학급 학생수,특수학급 학급당 학생수,학급수(계),학생수(계),학급당 학생수(계),교사수,수업교원 1인당 학생수,폐교연도
225,2014,서울특별시 금천구,S010001256,서울흥일초등학교,36.0,28.0,64.0,3,63,21.0,...,25.0,1,3,3.0,19,431,22.7,24,18.0,2015
422,2014,서울특별시 은평구,S010002020,알로이시오초등학교,0.0,0.0,0.0,0,0,0.0,...,17.7,2,12,6.0,10,145,14.5,10,14.5,2015
421,2017,서울특별시 은평구,S010002022,은혜초등학교,28.0,16.0,44.0,2,44,22.0,...,14.0,0,0,0.0,12,254,21.2,12,21.2,2018
146,2019,서울특별시 강서구,S010001047,서울염강초등학교,5.0,8.0,13.0,1,13,13.0,...,19.5,2,7,3.5,12,157,13.1,15,10.5,2020
16,2022,서울특별시 서초구,S010000754,서울반포초등학교,0.0,0.0,0.0,0,0,0.0,...,18.1,0,0,0.0,16,215,13.4,19,11.3,2023
454,2022,서울특별시 광진구,S010002195,서울화양초등학교,4.0,3.0,7.0,1,7,7.0,...,9.0,0,0,0.0,8,84,10.5,11,7.6,2023
234,2023,서울특별시 중랑구,S010001407,서울면중초등학교,37.0,36.0,73.0,3,69,23.0,...,14.7,1,5,5.0,16,320,20.0,19,16.8,2024
510,2024,서울특별시 용산구,S010002379,서울보광초등학교,10.0,11.0,21.0,3,31,10.3,...,15.0,2,9,4.5,15,158,10.5,21,7.5,2025


In [14]:
# 폐교 학교 코드 리스트
closed_codes = closed_schools_all['정보공시 학교코드'].unique()

# merged_data에서 폐교 학교 제외
merged_data_filtered = merged_data[~merged_data['정보공시 학교코드'].isin(closed_codes)]

print(merged_data_filtered.shape)

(10079, 33)


In [15]:
print(merged_data.shape)

(10170, 33)


In [16]:
merged_data_filtered.columns

Index(['년도', '지역', '정보공시 학교코드', '학교명', '입학생_남', '입학생_여', '입학생수', '1학년 학급수',
       '1학년 학생수', '1학년 학급당 학생수', '2학년 학급수', '2학년 학생수', '2학년 학급당 학생수',
       '3학년 학급수', '3학년 학생수', '3학년 학급당 학생수', '4학년 학급수', '4학년 학생수',
       '4학년 학급당 학생수', '5학년 학급수', '5학년 학생수', '5학년 학급당 학생수', '6학년 학급수',
       '6학년 학생수', '6학년 학급당 학생수', '특수학급 학급수', '특수학급 학생수', '특수학급 학급당 학생수',
       '학급수(계)', '학생수(계)', '학급당 학생수(계)', '교사수', '수업교원 1인당 학생수'],
      dtype='object')

In [104]:
location = pd.read_csv("./data/학교기본정보(초)_서울특별시교육청.csv")
loacation_= location[['정보공시 학교코드','위도', '경도']]

In [105]:
# merged_data와 조인
merged_data = merged_data_filtered.merge(loacation_, on='정보공시 학교코드', how='left')

# 확인
print(merged_data.head())


     년도         지역   정보공시 학교코드              학교명  입학생_남  입학생_여   입학생수  1학년 학급수  \
0  2009  서울특별시 서초구  S000003511    서울교육대학교부설초등학교   60.0   60.0  120.0        5   
1  2009  서울특별시 종로구  S000003563  서울대학교사범대학부설초등학교   53.0   51.0  104.0        5   
2  2009  서울특별시 강남구  S010000737         서울개원초등학교   58.0   63.0  121.0        7   
3  2009  서울특별시 강남구  S010000738         서울개일초등학교   47.0   50.0   97.0        5   
4  2009  서울특별시 강남구  S010000739         서울개포초등학교   25.0   17.0   42.0        4   

   1학년 학생수  1학년 학급당 학생수  ...  특수학급 학급수  특수학급 학생수  특수학급 학급당 학생수  학급수(계)  \
0      120         24.0  ...         5         0           0.0      35   
1      104         20.8  ...         5         0           0.0      35   
2      121         17.3  ...         7         0           0.0      49   
3       97         19.4  ...         5         0           0.0      35   
4       42         10.5  ...         4         2           0.5      28   

   학생수(계)  학급당 학생수(계)  교사수  수업교원 1인당 학생수         위도          경도  
0 

In [ ]:
# pip install pystan==2.19.1.1

Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\김다은\Desktop\data_anal_port\streamlit_env\Scripts\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-b814d5m6\\pystan_531a7a565d3649179c5da882d4d26ad2\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-b814d5m6\\pystan_531a7a565d3649179c5da882d4d26ad2\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-pip-egg-info-23bkh4pk'
         cwd: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-b814d5m6\pystan_531a7a565d3649179c5da882d4d26ad2\
    Complete output (5 lines):
    Traceback (most 

In [ ]:
# pip install prophet

  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\김다은\Desktop\data_anal_port\streamlit_env\Scripts\python.exe -m pip install --upgrade pip' command.


In [24]:
import pandas as pd
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# =======================
# 1. 데이터 준비
# =======================
df = merged_data[['년도','정보공시 학교코드','학교명','입학생수','학급수(계)','교사수','학생수(계)']]

# 학습용: 2024년까지
train = df[df['년도'] <= 2024]
test  = df[df['년도'] == 2025]

# =======================
# 2. Prophet (시계열 예측) - 학교별 반복
# =======================
prophet_preds = []

# 2025년 데이터 기준으로 학교코드 순서 정렬
test_sorted = test.sort_values('정보공시 학교코드')
test_schools = test_sorted['정보공시 학교코드'].tolist()

for code in test_schools:
    train_school = train[train['정보공시 학교코드'] == code][['년도','학생수(계)']]
    
    if train_school.shape[0] < 2:  # 데이터가 너무 적으면 예측 불가
        prophet_preds.append(np.nan)
        continue
    
    prophet_df = train_school.rename(columns={'년도':'ds','학생수(계)':'y'})
    model = Prophet(yearly_seasonality=False, daily_seasonality=False)
    model.fit(prophet_df)
    
    future = pd.DataFrame({'ds':[2025]})
    forecast = model.predict(future)
    prophet_preds.append(forecast['yhat'].values[0])

# =======================
# 3. RandomForest (회귀 예측)
# =======================
X_train = train[['입학생수','학급수(계)','교사수']]
y_train = train['학생수(계)']

X_test = test[['입학생수','학급수(계)','교사수']]
y_test = test['학생수(계)']

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

# =======================
# 4. 정확도 비교
# =======================
# Prophet 예측 중 nan 제거
valid_idx = ~np.isnan(prophet_preds)
rmse_prophet = np.sqrt(mean_squared_error(y_test.values[valid_idx], np.array(prophet_preds)[valid_idx]))
mae_prophet  = mean_absolute_error(y_test.values[valid_idx], np.array(prophet_preds)[valid_idx])

rmse_rf = np.sqrt(mean_squared_error(y_test, rf_pred))
mae_rf  = mean_absolute_error(y_test, rf_pred)

print("Prophet → RMSE:", rmse_prophet, " MAE:", mae_prophet)
print("RandomForest → RMSE:", rmse_rf, " MAE:", mae_rf)



23:13:30 - cmdstanpy - INFO - Chain [1] start processing
23:13:30 - cmdstanpy - INFO - Chain [1] done processing
23:13:30 - cmdstanpy - INFO - Chain [1] start processing
23:13:30 - cmdstanpy - INFO - Chain [1] done processing
23:13:30 - cmdstanpy - INFO - Chain [1] start processing
23:13:31 - cmdstanpy - INFO - Chain [1] done processing
23:13:31 - cmdstanpy - INFO - Chain [1] start processing
23:13:31 - cmdstanpy - INFO - Chain [1] done processing
23:13:31 - cmdstanpy - INFO - Chain [1] start processing
23:13:31 - cmdstanpy - INFO - Chain [1] done processing
23:13:31 - cmdstanpy - INFO - Chain [1] start processing
23:13:32 - cmdstanpy - INFO - Chain [1] done processing
23:13:32 - cmdstanpy - INFO - Chain [1] start processing
23:13:32 - cmdstanpy - INFO - Chain [1] done processing
23:13:32 - cmdstanpy - INFO - Chain [1] start processing
23:13:32 - cmdstanpy - INFO - Chain [1] done processing
23:13:32 - cmdstanpy - INFO - Chain [1] start processing
23:13:33 - cmdstanpy - INFO - Chain [1]

Prophet → RMSE: 146.65557428932925  MAE: 90.38284381305063
RandomForest → RMSE: 82.84427846341265  MAE: 53.95611839959469


In [106]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# =======================
# 1. 데이터 준비
# =======================
df = merged_data[['년도','정보공시 학교코드','학교명','지역','입학생수','학급수(계)','교사수','학생수(계)']]

# 학습용: 2024년까지
train = df[df['년도'] <= 2024]
# 테스트용: 2025년
test  = df[df['년도'] == 2025]

# =======================
# 2. 범주형 변수 원-핫 인코딩 (지역)
# =======================
train = pd.get_dummies(train, columns=['지역'])
test  = pd.get_dummies(test, columns=['지역'])

# 학습/예측용 피처
feature_cols = ['입학생수','학급수(계)','교사수','년도'] + [c for c in train.columns if c.startswith('지역_')]

X_train = train[feature_cols]
y_train = train['학생수(계)']

X_test = test[feature_cols]
y_test = test['학생수(계)']

# =======================
# 3. RandomForest 모델 학습
# =======================
rf = RandomForestRegressor(
    n_estimators=300,   # 나무 300개
    max_depth=15,       # 최대 깊이 15
    random_state=42,
    n_jobs=-1           # CPU 코어 모두 사용
)
rf.fit(X_train, y_train)

# =======================
# 4. 2025년 예측
# =======================
rf_pred = rf.predict(X_test)

# =======================
# 5. 정확도 평가
# =======================
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_pred))
mae_rf  = mean_absolute_error(y_test, rf_pred)

print("RandomForest → RMSE:", rmse_rf, " MAE:", mae_rf)

# =======================
# 6. 예측값 확인 (원하면)
# =======================
pred_df = test[['정보공시 학교코드','학교명']].copy()
pred_df['예측_학생수'] = rf_pred
pred_df['실제_학생수'] = y_test.values

pred_df.head(10)


RandomForest → RMSE: 56.94042716326833  MAE: 37.32007490886093


,정보공시 학교코드,학교명,예측_학생수,실제_학생수
9473,S000003511,서울교육대학교부설초등학교,606.635893,611
9474,S000003563,서울대학교사범대학부설초등학교,598.309369,570
9475,S010000737,서울개원초등학교,1006.558612,1006
9476,S010000738,서울개일초등학교,904.134572,834
9477,S010000739,서울개포초등학교,930.227151,886
9478,S010000741,서울구룡초등학교,515.999924,561
9479,S010000742,서울논현초등학교,198.011929,198
9480,S010000743,서울대곡초등학교,943.670083,1235
9481,S010000744,서울대도초등학교,1667.902122,1956
9482,S010000745,서울대모초등학교,620.687723,788


RMSE (Root Mean Squared Error)

예측값과 실제값의 차이를 제곱해 평균한 뒤 제곱근

값이 크면 큰 오차가 있다는 의미

56 정도면, 한 학교의 학생수 예측에서 평균적으로 ±56명 정도 오차가 있다는 의미

MAE (Mean Absolute Error)

단순 절대 오차 평균

37 정도면 한 학교의 학생수 예측에서 평균적으로 ±37명 정도 차이

In [107]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =======================
# 1. 데이터 준비
# =======================
df = merged_data[['년도','정보공시 학교코드','학교명','지역','입학생수','학생수(계)','교사수']]

# 학습용: 2024년까지
train = df[df['년도'] <= 2024].copy()
# 테스트용: 2025년
test  = df[df['년도'] == 2025].copy()

# =======================
# 2. 지역 원-핫 인코딩
# =======================
train = pd.get_dummies(train, columns=['지역'])
test  = pd.get_dummies(test, columns=['지역'])

# =======================
# 3. 내년도 타겟 생성 (shift)
# =======================
train_sorted = train.sort_values(['정보공시 학교코드','년도'])
for col in ['입학생수','학생수(계)','교사수']:
    train_sorted[col+'_next'] = train_sorted.groupby('정보공시 학교코드')[col].shift(-1)

# 내년도 값이 있는 행만 사용
train_clean = train_sorted.dropna(subset=['입학생수_next','학생수(계)_next','교사수_next'])

feature_cols = ['입학생수','학생수(계)','교사수'] + [c for c in train.columns if c.startswith('지역_')]
target_cols = ['입학생수_next','학생수(계)_next','교사수_next']

X_train = train_clean[feature_cols]
y_train = train_clean[target_cols]

# =======================
# 4. MultiOutput RandomForest 학습
# =======================
rf_multi = MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
rf_multi.fit(X_train, y_train)

# =======================
# 5. 2025년 예측
# =======================
X_test = test[feature_cols]
y_test = test[['입학생수','학생수(계)','교사수']].values

y_pred = rf_multi.predict(X_test)

# =======================
# 6. 정확도 평가
# =======================
rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
mae  = mean_absolute_error(y_test, y_pred)

print("RandomForest (MultiOutput) → RMSE (입학생수, 학생수, 교사수):", rmse)
print("RandomForest (MultiOutput) → MAE (전체):", mae)

# =======================
# 7. 예측값 확인
# =======================
pred_df = test[['학교명','정보공시 학교코드']].copy()
pred_df[['입학생수_예측','학생수_예측','교사수_예측']] = y_pred
pred_df[['입학생수_실제','학생수_실제','교사수_실제']] = y_test
pred_df.head(10)


RandomForest (MultiOutput) → RMSE (입학생수, 학생수, 교사수): [11.62402959 53.31821017  3.54739491]
RandomForest (MultiOutput) → MAE (전체): 15.354328897969161


,학교명,정보공시 학교코드,입학생수_예측,학생수_예측,교사수_예측,입학생수_실제,학생수_실제,교사수_실제
9473,서울교육대학교부설초등학교,S000003511,95.785,610.485,31.345,96.0,611.0,31.0
9474,서울대학교사범대학부설초등학교,S000003563,82.590,532.785,33.525,87.0,570.0,36.0
9475,서울개원초등학교,S010000737,149.120,959.265,49.060,149.0,1006.0,51.0
9476,서울개일초등학교,S010000738,117.010,796.460,39.745,129.0,834.0,40.0
9477,서울개포초등학교,S010000739,116.530,859.990,49.375,115.0,886.0,51.0
9478,서울구룡초등학교,S010000741,74.165,667.620,37.510,70.0,561.0,32.0
9479,서울논현초등학교,S010000742,26.890,178.405,16.680,35.0,198.0,17.0
9480,서울대곡초등학교,S010000743,121.270,1229.765,52.730,108.0,1235.0,52.0
9481,서울대도초등학교,S010000744,208.300,1933.315,74.290,210.0,1956.0,73.0
9482,서울대모초등학교,S010000745,91.505,719.710,36.025,90.0,788.0,37.0


In [108]:
future_years = [2025, 2026, 2027, 2028, 2029]
pred_list = []

# 초기 입력: 2024년 데이터
current_df = df[df['년도']==2024].copy()

for year in future_years:
    # 지역 원-핫 인코딩
    current_input = pd.get_dummies(current_df, columns=['지역'])
    
    # 피처 선택
    X_current = current_input[feature_cols]
    
    # 예측
    y_pred = rf_multi.predict(X_current)
    
    # 결과 저장
    pred_df = current_df[['정보공시 학교코드','학교명','지역']].copy()
    pred_df[['입학생수','학생수','교사수']] = y_pred
    pred_df['년도'] = year
    pred_list.append(pred_df)
    
    # 다음년도 입력 업데이트
    current_df[['입학생수','학생수(계)','교사수']] = y_pred

# 5년치 예측 데이터 결합
future_5yrs_df = pd.concat(pred_list, ignore_index=True)


# 미래예측

In [109]:
future_5yrs_df

,정보공시 학교코드,학교명,지역,입학생수,학생수,교사수,년도
0,S000003511,서울교육대학교부설초등학교,서울특별시 서초구,95.850,610.720,31.595,2025
1,S000003563,서울대학교사범대학부설초등학교,서울특별시 종로구,79.655,531.960,33.770,2025
2,S010000737,서울개원초등학교,서울특별시 강남구,163.700,908.700,40.185,2025
3,S010000738,서울개일초등학교,서울특별시 강남구,124.100,888.600,42.925,2025
4,S010000739,서울개포초등학교,서울특별시 강남구,133.840,867.810,46.695,2025
...,...,...,...,...,...,...,...
3015,S010006432,서울항동초등학교,서울특별시 구로구,205.630,1276.375,66.280,2029
3016,S010006433,서울해누리초등학교,서울특별시 송파구,103.840,607.125,33.800,2029
3017,S010006476,서울양원숲초등학교,서울특별시 중랑구,106.815,614.520,36.435,2029
3018,S010006510,서울개현초등학교,서울특별시 강남구,111.875,691.530,34.355,2029


In [113]:
# 수업교원 1인당 학생수 = 학생수(계) / 교사수
future_5yrs_df['수업교원 1인당 학생수'] = future_5yrs_df['학생수'] / future_5yrs_df['교사수']

In [114]:
data_2025 = merged_data[merged_data['년도']==2025][['지역','학교명','입학생수','학생수(계)','학급당 학생수(계)','수업교원 1인당 학생수']]
data_2025

,지역,학교명,입학생수,학생수(계),학급당 학생수(계),수업교원 1인당 학생수
9473,서울특별시 서초구,서울교육대학교부설초등학교,96.0,611,21.8,19.7
9474,서울특별시 종로구,서울대학교사범대학부설초등학교,87.0,570,18.4,15.8
9475,서울특별시 강남구,서울개원초등학교,149.0,1006,22.9,19.7
9476,서울특별시 강남구,서울개일초등학교,129.0,834,25.3,20.9
9477,서울특별시 강남구,서울개포초등학교,115.0,886,20.6,17.4
...,...,...,...,...,...,...
10074,서울특별시 구로구,서울항동초등학교,276.0,1478,21.1,18.2
10075,서울특별시 송파구,서울해누리초등학교,101.0,700,21.9,18.4
10076,서울특별시 중랑구,서울양원숲초등학교,172.0,779,21.6,17.7
10077,서울특별시 강남구,서울개현초등학교,130.0,800,23.5,21.6


In [115]:
# 1분위수 (25%) 계산
q1 = data_2025.groupby('지역')[['입학생수','학생수(계)','학급당 학생수(계)','수업교원 1인당 학생수']].quantile(0.25)

print(q1)


             입학생수  학생수(계)  학급당 학생수(계)  수업교원 1인당 학생수
지역                                                 
서울특별시 강남구   50.50  419.25      18.525        15.900
서울특별시 강동구   81.00  550.00      18.600        15.500
서울특별시 강북구   57.00  415.00      17.025        14.575
서울특별시 강서구   46.00  355.00      17.350        14.000
서울특별시 관악구   38.25  262.50      16.225        13.650
서울특별시 광진구   43.00  332.00      17.000        13.900
서울특별시 구로구   47.00  361.75      16.600        13.750
서울특별시 금천구   32.25  235.00      15.750        11.675
서울특별시 노원구   33.50  276.50      15.675        13.325
서울특별시 도봉구   37.00  301.00      17.150        14.150
서울특별시 동대문구  49.00  352.00      15.800        13.300
서울특별시 동작구   60.00  398.00      17.000        14.000
서울특별시 마포구   54.50  387.00      17.875        14.425
서울특별시 서대문구  48.00  352.00      17.750        14.400
서울특별시 서초구   66.00  459.00      19.550        16.500
서울특별시 성동구   36.00  202.00      14.500        12.600
서울특별시 성북구   52.00  369.00      17.600        14.200
서울특별시 송파구   

초등학교 기준

OECD 평균: 약 15~20명/교사

한국 교육부 권장: 1인당 학생수 20명 내외

In [116]:
years = [2026, 2027, 2028, 2029]
all_outlier_schools = []
already_closed_codes = set()  # 이전 연도에 폐교된 학교 코드 저장

cols = ['입학생수','학생수','수업교원 1인당 학생수']

for year in years:
    # 해당 연도 데이터 선택
    data_year = future_5yrs_df[future_5yrs_df['년도'] == year].copy()
    
    # 이미 폐교된 학교 제외
    data_year = data_year[~data_year['정보공시 학교코드'].isin(already_closed_codes)]
    
    # 이상치 판정용 DataFrame 초기화
    outlier_flags = pd.DataFrame(False, index=data_year.index, columns=cols)
    
    # 각 컬럼별 지역 기준 10% 이하 판정
    for col in cols:
        # 지역별 10% 분위수 계산
        q10_by_region = data_year.groupby('지역')[col].transform(lambda x: x.quantile(0.05))
        # 10% 이하인 경우 True
        outlier_flags[col] = data_year[col] <= q10_by_region
    
    # 세 컬럼 모두 10% 이하인 학교 선택
    outlier_schools = data_year[outlier_flags.all(axis=1)].copy()
    outlier_schools['년도'] = year  # 연도 표시
    
    # 이미 폐교된 학교 코드 업데이트
    already_closed_codes.update(outlier_schools['정보공시 학교코드'].tolist())
    
    # 리스트에 추가
    all_outlier_schools.append(outlier_schools)

# 2026~2029년 폐교 후보 합치기
outlier_schools_2026_2029 = pd.concat(all_outlier_schools, ignore_index=True)


# 2026~ 2029 년도 폐교 후보 리스트
- 당시 자치구 기준으로 입학생수, 재학생수, 수업교원 1인당 학생수 칼럼이 하위 5 프로 이하 인 경우 해당 학교는 폐교 위기에 처한다.

In [117]:
outlier_schools_2026_2029

,정보공시 학교코드,학교명,지역,입학생수,학생수,교사수,년도,수업교원 1인당 학생수
0,S010000747,서울대진초등학교,서울특별시 강남구,30.005,201.035,17.600,2026,11.422443
1,S010000757,서울방현초등학교,서울특별시 서초구,28.955,191.110,17.235,2026,11.088483
2,S010000778,서울우암초등학교,서울특별시 서초구,31.475,227.525,17.130,2026,13.282253
3,S010000910,서울남천초등학교,서울특별시 송파구,40.645,241.360,21.090,2026,11.444286
4,S010000923,서울상일초등학교,서울특별시 강동구,29.685,160.050,14.210,2026,11.263195
...,...,...,...,...,...,...,...,...
65,S010001785,서울창원초등학교,서울특별시 도봉구,33.440,216.175,17.155,2029,12.601282
66,S010002312,서울안암초등학교,서울특별시 성북구,42.285,230.275,19.795,2029,11.632988
67,S010002324,서울청덕초등학교,서울특별시 성북구,41.565,233.230,19.555,2029,11.926873
68,S010002405,운현초등학교,서울특별시 종로구,30.020,169.565,14.305,2029,11.853548
